import sys  
reload(sys)  
sys.setdefaultencoding('utf-8')

In [46]:
import numpy.linalg as LA
import matplotlib.pyplot as plt
#from my_pickle import my_pickle_load, my_pickle_dump

%matplotlib inline

In [47]:
from nltk.tokenize.stanford_segmenter import StanfordSegmenter

segmenter = StanfordSegmenter(path_to_jar='./stanford-segmenter-2014-08-27/stanford-segmenter-3.4.1.jar', 
                              path_to_slf4j='./stanford-segmenter-2014-08-27/slf4j-api.jar',
                              path_to_sihan_corpora_dict='./stanford-segmenter-2014-08-27/data', 
                              path_to_model='./stanford-segmenter-2014-08-27/data/pku.gz', 
                              path_to_dict='./stanford-segmenter-2014-08-27/data/dict-chris6.ser.gz')

In [48]:
segmenter = StanfordSegmenter(path_to_jar='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/stanford-segmenter-3.4.1.jar', 
                              path_to_slf4j='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/slf4j-api.jar',
                              path_to_sihan_corpora_dict='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/data', 
                              path_to_model='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/data/pku.gz', 
                              path_to_dict='/Users/nhu2000/desktop/nltk/stanford-segmenter-2014-08-27/data/dict-chris6.ser.gz')

str=u"我在博客园开了一个博客，我的博客名叫伏草惟存，写了一些自然语言处理的文章。"
result = segmenter.segment(str)

print result

In [49]:
import cPickle as pickle
import pandas as pd
import numpy as np

In [50]:
import string
import math
#import glob
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk import tokenize
from nltk.stem.porter import *
from nltk.corpus import stopwords

from gensim import matutils
from gensim.models.ldamodel import LdaModel
from gensim.models.lsimodel import LsiModel

from gensim import corpora, models, similarities
from sklearn.metrics.pairwise import linear_kernel


In [51]:
import re
from urlparse import urlparse
topHostPostfix = (
    '.com','.la','.io','.co','.info','.net','.org','.me','.mobi',
    '.us','.biz','.xxx','.ca','.co.jp','.com.cn','.net.cn',
    '.org.cn','.mx','.tv','.ws','.ag','.com.ag','.net.ag',
    '.org.ag','.am','.asia','.at','.be','.com.br','.net.br',
    '.bz','.com.bz','.net.bz','.cc','.com.co','.net.co',
    '.nom.co','.de','.es','.com.es','.nom.es','.org.es',
    '.eu','.fm','.fr','.gs','.in','.co.in','.firm.in','.gen.in',
    '.ind.in','.net.in','.org.in','.it','.jobs','.jp','.ms',
    '.com.mx','.nl','.nu','.co.nz','.net.nz','.org.nz',
    '.se','.tc','.tk','.tw','.com.tw','.idv.tw','.org.tw',
    '.hk','.co.uk','.me.uk','.org.uk','.vg', ".com.hk")
regx = r'[^\.]+('+'|'.join([h.replace('.',r'\.') for h in topHostPostfix])+')$'
pattern = re.compile(regx,re.IGNORECASE)

In [52]:
def my_pickle_dump(fname, data):
    with open(fname, 'w') as f:
        pickle.dump(data, f)

def my_pickle_load(fname):
    with open(fname) as f:
        return pickle.load(f)

In [102]:
df = pd.read_csv('/Users/nhu2000/desktop/huizhou/data/nutch-crawl-huizhou-domain_only2-02-15-2017-cleaned.csv')

In [111]:
df.head()

,id,tstamp,url,domain,title,digest,content
0,http://cul.anhuinews.com/system/2009/03/05/002...,2017-02-13T17:25:31.777Z,http://cul.anhuinews.com/system/2009/03/05/002...,anhuinews.com,"第十六章(2)-月上重火,连载,阅读-中安在线-徽文化",306a2bbf507dca8af08ca26bdc835949,"第十六章(2)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ..."
1,http://cul.anhuinews.com/system/2009/03/05/002...,2017-02-13T17:47:03.783Z,http://cul.anhuinews.com/system/2009/03/05/002...,anhuinews.com,"第十六章(1)-月上重火,连载,阅读-中安在线-徽文化",38d30b20526082bbcdfc2d911528da3a,"第十六章(1)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ..."
2,http://cul.anhuinews.com/system/2009/03/05/002...,2017-02-13T16:20:40.982Z,http://cul.anhuinews.com/system/2009/03/05/002...,anhuinews.com,"第十五章(4)-月上重火,连载,阅读-中安在线-徽文化",c80267d57e6a8c6eecb593b2617e438c,"第十五章(4)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ..."
3,http://cul.anhuinews.com/system/2009/03/05/002...,2017-02-13T18:57:11.49Z,http://cul.anhuinews.com/system/2009/03/05/002...,anhuinews.com,"第十五章(3)-月上重火,连载,阅读-中安在线-徽文化",31ea7eaa5e81e56ed3fc2a54e3978530,"第十五章(3)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ..."
4,http://cul.anhuinews.com/system/2009/03/05/002...,2017-02-13T16:40:17.225Z,http://cul.anhuinews.com/system/2009/03/05/002...,anhuinews.com,"第十五章(2)-月上重火,连载,阅读-中安在线-徽文化",a22abfd3a1ed3d6065f9fb3ec2502e3f,"第十五章(2)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ..."


In [112]:
def get_columns():
    return ('dt', 'from', 'host', 'domain', 'url', 'title', 't_tokens', 't_len', 'summary', 'image', 'abstract', \
            'a_tokens', 'a_len', 'content', 'c_tokens', 'c_len') 
def get_empty_columns():
    return ('', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '') 

In [113]:
df_0 = pd.DataFrame(columns = get_columns())

def map_host(x):
    t0 = x.split('://')
    t1 = t0[1].split('/')
    return t1[0]

def map_host(x):
    t0 = x.split('://')[1].split('/')[0]
    return t0

print map_host('http://www.99huizhou.com/thread-102832-1-1.html')


In [114]:
df_0['dt'] = df['tstamp']
df_0['url'] = df['url']
df_0['title'] = df['title']
df_0['content'] = df['content']
df_0['domain'] = df['domain']
df_0['host'] = df.url.map(lambda x: x.split('://')[1].split('/')[0] )  


In [115]:
df_0.head()

,dt,from,host,domain,url,title,t_tokens,t_len,summary,image,abstract,a_tokens,a_len,content,c_tokens,c_len
0,2017-02-13T17:25:31.777Z,NaN,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/system/2009/03/05/002...,"第十六章(2)-月上重火,连载,阅读-中安在线-徽文化",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"第十六章(2)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ...",NaN,NaN
1,2017-02-13T17:47:03.783Z,NaN,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/system/2009/03/05/002...,"第十六章(1)-月上重火,连载,阅读-中安在线-徽文化",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"第十六章(1)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ...",NaN,NaN
2,2017-02-13T16:20:40.982Z,NaN,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/system/2009/03/05/002...,"第十五章(4)-月上重火,连载,阅读-中安在线-徽文化",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"第十五章(4)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ...",NaN,NaN
3,2017-02-13T18:57:11.49Z,NaN,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/system/2009/03/05/002...,"第十五章(3)-月上重火,连载,阅读-中安在线-徽文化",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"第十五章(3)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ...",NaN,NaN
4,2017-02-13T16:40:17.225Z,NaN,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/system/2009/03/05/002...,"第十五章(2)-月上重火,连载,阅读-中安在线-徽文化",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"第十五章(2)-月上重火,连载,阅读-中安在线-徽文化 首页 新闻 安徽 网视 ...",NaN,NaN


In [116]:
cnt = Counter(df_0['domain'].values)

In [117]:
print len(cnt)

4


In [118]:
most_common_cnt = cnt.most_common(1177)
print most_common_cnt

[('ahage.net', 15465), ('anhuinews.com', 5474), ('99huizhou.com', 337), ('517huizhou.com', 5)]


http://cul.anhuinews.com   
http://www.hzwh.com   (!!!)  slow
http://www.517huizhou.com  (!!!)
http://www.99huizhou.com
http://www.ahage.net
http://www.xinhs.cn/toplistb.asp?id=870 (!!!)

In [62]:
cnt = Counter(df_0['host'].values)

In [120]:
most_common_cnt = cnt.most_common(150)
print most_common_cnt

[('ahage.net', 15465), ('anhuinews.com', 5474), ('99huizhou.com', 337), ('517huizhou.com', 5)]


In [121]:
domain_unique = df_0['domain'].unique()

In [122]:
print len(domain_unique)

4


In [123]:
print domain_unique

['anhuinews.com' 'ahage.net' '99huizhou.com' '517huizhou.com']


In [124]:
host_unique = df_0['host'].unique()

In [125]:
print len(host_unique)

4


In [126]:
print host_unique

['cul.anhuinews.com' 'www.ahage.net' 'www.99huizhou.com'
 'www.517huizhou.com']


In [127]:
df_0.to_csv('temp001.csv', encoding='utf-8', index=False)

In [128]:
new_df = pd.read_csv('temp001.csv')

In [ ]:
for i in range(0, len(new_df)):
    t0 = new_df.loc[i]['title']
    t1 = new_df.loc[i]['domain']
    if t1 == '99huizhou.com':
        t0 = ''
    elif t1 == '517huizhou.com'
        t0 = ''
    else:
        print i, new_df.loc[i]['domain']
        t0 = new_df.loc[i]['title']
        t1 = new_df.loc[i]['domain']
        if '徽文化频道' in t0:
            t0 = t0.replace('徽文化频道', '') 
            print t0[:-28]
        if '中安在线' in t0 :
            t0 = t0.replace('中安在线', '') 
            print '中安在线----', t0[:-23]
        elif 'www.ahage.net' in t0:
            if '安徽文化网' in t0:
                t0 = t0.replace('安徽文化网', '') 
            if 'www.ahage.net' in t0:
                t0 = t0.replace('www.ahage.net', '') 
            if '我的网站' in t0:
                t0 = t0.replace('我的网站', '') 
            print 'www.ahage.net----', t0
        elif 'ahage.net' in t0:
            if '| 安徽文化人的网上家园，网上最大的安徽历史文化资料中心' in t0:
                t0 = t0.replace('| 安徽文化人的网上家园，网上最大的安徽历史文化资料中心', '')
            if '安徽文化网' in t0:
                t0 = t0.replace('安徽文化网', '') 
            if 'ahage.net' in t0:
                t0 = t0.replace('ahage.net', '') 
            if '_我的网站' in t0:
                t0 = t0.replace('_我的网站', '') 
            print 'ahage.net----', t0
        else:
            if '我的网站' in t0:
                t0 = t0.replace('我的网站', '')             
        
        t0 

In [98]:
cnt = 0
for item in df_0['title']:
    cnt += 1
    if (cnt > 8900 and cnt < 9000):
        print cnt, '------------'
        t0 = item.split('编辑信箱：')
        if len(t0) > 1:
            t1 = t0[1].split(' ')
            t2 = t0[1].split('欢迎进入文学情感社区讨论>>>')
            if len(t2[0][len(t1[0]):]) > 200:
                t3 = t2[0][len(t1[0]):].lstrip()
                t4 = t3.split(' ')[1]
                print len(t3), t4, t3
            else:
                print "----------- None"
                print item
        else:
            print "----------- Other"
            print item

8901 ------------
----------- Other
天长市：送孝文化扬剧《千里寻母》助爱心募捐活动_我的网站
8902 ------------
----------- Other
芜湖县文化馆和部分群文辅导员受省文化厅表彰_我的网站
8903 ------------
----------- Other
六安市广播电视台及《皖西日报》到金寨县采访广播村村响工程_我的网站
8904 ------------
----------- Other
凤阳县委常委陈开广调研乡镇综合文化站_安徽文化网 | 安徽文化人的网上家园，网上最大的安徽历史文化资料中心 ahage.net
8905 ------------
----------- Other
凤阳县人大深入殷涧镇调研文化民生工作_安徽文化网 | 安徽文化人的网上家园，网上最大的安徽历史文化资料中心 ahage.net
8906 ------------
----------- Other
安广“映山红行动”美术志愿者到金寨县桃岭乡_安徽文化网 | 安徽文化人的网上家园，网上最大的安徽历史文化资料中心 ahage.net
8907 ------------
----------- Other
爱心无止境，众筹见真情_安徽文化网 | 安徽文化人的网上家园，网上最大的安徽历史文化资料中心 ahage.net
8908 ------------
----------- Other
话剧下乡传承文化精粹，志愿服务奉献无私爱心_安徽文化网 | 安徽文化人的网上家园，网上最大的安徽历史文化资料中心 ahage.net
8909 ------------
----------- Other
金寨桃岭养生小院纳客忙_安徽文化网 | 安徽文化人的网上家园，网上最大的安徽历史文化资料中心 ahage.net
8910 ------------
----------- Other
大力支持，助力爱心_安徽文化网 | 安徽文化人的网上家园，网上最大的安徽历史文化资料中心 ahage.net
8911 ------------
----------- Other
身处惠丰灵秀之景 传递你我微薄爱心_安徽文化网 | 安徽文化人的网上家园，网上最大的安徽历史文化资料中心 ahage.net
8912 ------------
--

In [101]:
cnt = 0
for item in df_0['title']:
    cnt += 1
    if (cnt > 9900 and cnt < 10000):
        print cnt, '------------'
        print item


9901 ------------
第一章 湘西列车(5)-凤凰面具,蘑菇,连载,阅读-中安在线-徽文化
9902 ------------
第一章 湘西列车(3)-凤凰面具,蘑菇,连载,阅读-中安在线-徽文化
9903 ------------
逃犯目击报告(10)-越狱,保罗・卢迪提斯,重庆出版社-中安在线-徽文化
9904 ------------
逃犯目击报告(9)-越狱,保罗・卢迪提斯,重庆出版社-中安在线-徽文化
9905 ------------
逃犯目击报告(7)-越狱,保罗・卢迪提斯,重庆出版社-中安在线-徽文化
9906 ------------
逃犯目击报告(6)-越狱,保罗・卢迪提斯,重庆出版社-中安在线-徽文化
9907 ------------
第六节(2)-生死线,兰晓龙,连载,阅读-中安在线-徽文化
9908 ------------
徽商工业创始人胡练九-胡练九简介,徽商-中安在线-徽文化
9909 ------------
程十万-徽商,程十万简介-中安在线-徽文化
9910 ------------
现代著名民族资本家程霖生-徽商,程霖生简介-中安在线-徽文化
9911 ------------
第三十二章・鸡蛋汤的后遗症(3)-有一种爱情叫兄弟,连载,阅读-中安在线-徽文化
9912 ------------
怀　念　钢　笔-原创文学-中安在线-徽文化
9913 ------------
金瓶梅发源徽州 作者可能是徽州人-金瓶梅-中安在线-徽文化
9914 ------------
蚌埠市花鼓灯保护工作步伐进一步加快-花鼓灯-中安在线-徽文化
9915 ------------
徽剧的历史沿革-徽剧-中安在线-徽文化
9916 ------------
雾里青茶与瑞典哥德堡号的渊源-雾里青-中安在线-徽文化
9917 ------------
儒商梦绝：徽商潮落潮起-徽商-中安在线-徽文化
9918 ------------
腰缠十万贯的徽商哪里去了 -徽商-中安在线-徽文化
9919 ------------
明清文献中“徽商”一词的考察-徽商-中安在线-徽文化
9920 ------------
“茶马古道”能否成为安徽“黄山”？-茶马古道-中安在线-徽文化
9921 ------------
花鼓灯台湾舞风采交响乐韩国奏

In [84]:
print len(t0)

1


In [89]:
t = '许辉随笔集《和自己的夜晚单独在一起》出版 2012年10月21日11时22分'
t0 = t.split(' ')
print t0[1]

2012年10月21日11时22分


In [24]:
df_0.head()

,dt,from,host,domain,url,title,t_tokens,t_len,summary,image,abstract,a_tokens,a_len,content,c_tokens,c_len
0,2017-01-15T16:23:45.703Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-102832-1-1.html,【视频】白岳笔会之座谈会-白岳笔会-故园徽州网 - Powered by Discuz!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,【视频】白岳笔会之座谈会-白岳笔会-故园徽州网 - Powered by Discuz! 故...,NaN,NaN
1,2017-01-15T16:32:56.699Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-102832-3-1.html,【视频】白岳笔会之座谈会-白岳笔会-故园徽州网 - Powered by Discuz!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,【视频】白岳笔会之座谈会-白岳笔会-故园徽州网 - Powered by Discuz! 故...,NaN,NaN
2,2017-01-15T17:09:56.419Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-108113-1-1.html,这个怎么用？ - 徽州摄影旅游群 - 故园徽州 - Powered by Discuz!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,这个怎么用？ - 徽州摄影旅游群 - 故园徽州 - Powered by Discuz! 故...,NaN,NaN
3,2017-01-15T19:00:55.159Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-110501-1-1.html,黄山日报：石潭摄影基地建设快速推进 - 徽州摄影旅游群 - 故园徽州 - Powered b...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,黄山日报：石潭摄影基地建设快速推进 - 徽州摄影旅游群 - 故园徽州 - Powered b...,NaN,NaN
4,2017-01-15T16:19:47.112Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-117056-1-1.html,故园徽州9月9日签到记录贴-说鳖茶馆-故园徽州网 - Powered by Discuz!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,故园徽州9月9日签到记录贴-说鳖茶馆-故园徽州网 - Powered by Discuz! ...,NaN,NaN


In [25]:
df_0.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62061 entries, 0 to 62060
Data columns (total 16 columns):
dt          62061 non-null object
from        0 non-null object
host        62061 non-null object
domain      62061 non-null object
url         62061 non-null object
title       61836 non-null object
t_tokens    0 non-null object
t_len       0 non-null object
summary     0 non-null object
image       0 non-null object
abstract    0 non-null object
a_tokens    0 non-null object
a_len       0 non-null object
content     62061 non-null object
c_tokens    0 non-null object
c_len       0 non-null object
dtypes: object(16)
memory usage: 8.0+ MB


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62061 entries, 0 to 62060
Data columns (total 7 columns):
id         62061 non-null object
tstamp     62061 non-null object
url        62061 non-null object
domain     62061 non-null object
title      61836 non-null object
digest     62061 non-null object
content    62061 non-null object
dtypes: object(7)
memory usage: 3.8+ MB


parts = urlparse(url0)
host = parts.netloc
m = pattern.search(host)
res =  m.group() if m else host

#print res
ret_h = "" if not res else res

print len(ret_h)

#domain
domain_url = parsed_url.split('http://')
domain_url1 = domain_url[1]
domain_url2 = domain_url1.split('/')
print domain_url2[0]

In [27]:
df_0['c_len'] = df_0['content'].str.len()
df_0['t_len'] = df_0['title'].str.len()

In [28]:
df_0['t_len'].max(), df_0['c_len'].max()

(296.0, 74764)

In [29]:
df_0['t_len'].min(), df_0['c_len'].min()

(2.0, 2)

In [30]:
df_dropped = df_0[df_0['c_len'] <= 100]

In [31]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 1855 to 62054
Data columns (total 16 columns):
dt          252 non-null object
from        0 non-null object
host        252 non-null object
domain      252 non-null object
url         252 non-null object
title       198 non-null object
t_tokens    0 non-null object
t_len       198 non-null float64
summary     0 non-null object
image       0 non-null object
abstract    0 non-null object
a_tokens    0 non-null object
a_len       0 non-null object
content     252 non-null object
c_tokens    0 non-null object
c_len       252 non-null int64
dtypes: float64(1), int64(1), object(14)
memory usage: 33.5+ KB


for item in df_dropped['c_len']:
    print item

In [33]:
df_cleaned = df_0[df_0['c_len'] > 100]

In [34]:
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61809 entries, 0 to 62060
Data columns (total 16 columns):
dt          61809 non-null object
from        0 non-null object
host        61809 non-null object
domain      61809 non-null object
url         61809 non-null object
title       61638 non-null object
t_tokens    0 non-null object
t_len       61638 non-null float64
summary     0 non-null object
image       0 non-null object
abstract    0 non-null object
a_tokens    0 non-null object
a_len       0 non-null object
content     61809 non-null object
c_tokens    0 non-null object
c_len       61809 non-null int64
dtypes: float64(1), int64(1), object(14)
memory usage: 8.0+ MB


In [35]:
df_no_dup = df_cleaned.drop_duplicates(['title'])

In [36]:
df_no_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54952 entries, 0 to 62059
Data columns (total 16 columns):
dt          54952 non-null object
from        0 non-null object
host        54952 non-null object
domain      54952 non-null object
url         54952 non-null object
title       54951 non-null object
t_tokens    0 non-null object
t_len       54951 non-null float64
summary     0 non-null object
image       0 non-null object
abstract    0 non-null object
a_tokens    0 non-null object
a_len       0 non-null object
content     54952 non-null object
c_tokens    0 non-null object
c_len       54952 non-null int64
dtypes: float64(1), int64(1), object(14)
memory usage: 7.1+ MB


In [37]:
df_no_dup = df_no_dup.dropna(subset =['content'])

In [38]:
df_no_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54952 entries, 0 to 62059
Data columns (total 16 columns):
dt          54952 non-null object
from        0 non-null object
host        54952 non-null object
domain      54952 non-null object
url         54952 non-null object
title       54951 non-null object
t_tokens    0 non-null object
t_len       54951 non-null float64
summary     0 non-null object
image       0 non-null object
abstract    0 non-null object
a_tokens    0 non-null object
a_len       0 non-null object
content     54952 non-null object
c_tokens    0 non-null object
c_len       54952 non-null int64
dtypes: float64(1), int64(1), object(14)
memory usage: 7.1+ MB


df_no_dup1 = df_no_dup.drop_duplicates(['content'])


df_no_dup1.info()

In [39]:
df_no_dup.head()

,dt,from,host,domain,url,title,t_tokens,t_len,summary,image,abstract,a_tokens,a_len,content,c_tokens,c_len
0,2017-01-15T16:23:45.703Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-102832-1-1.html,【视频】白岳笔会之座谈会-白岳笔会-故园徽州网 - Powered by Discuz!,NaN,86,NaN,NaN,NaN,NaN,NaN,【视频】白岳笔会之座谈会-白岳笔会-故园徽州网 - Powered by Discuz! 故...,NaN,3936
2,2017-01-15T17:09:56.419Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-108113-1-1.html,这个怎么用？ - 徽州摄影旅游群 - 故园徽州 - Powered by Discuz!,NaN,78,NaN,NaN,NaN,NaN,NaN,这个怎么用？ - 徽州摄影旅游群 - 故园徽州 - Powered by Discuz! 故...,NaN,3585
3,2017-01-15T19:00:55.159Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-110501-1-1.html,黄山日报：石潭摄影基地建设快速推进 - 徽州摄影旅游群 - 故园徽州 - Powered b...,NaN,111,NaN,NaN,NaN,NaN,NaN,黄山日报：石潭摄影基地建设快速推进 - 徽州摄影旅游群 - 故园徽州 - Powered b...,NaN,6176
4,2017-01-15T16:19:47.112Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-117056-1-1.html,故园徽州9月9日签到记录贴-说鳖茶馆-故园徽州网 - Powered by Discuz!,NaN,85,NaN,NaN,NaN,NaN,NaN,故园徽州9月9日签到记录贴-说鳖茶馆-故园徽州网 - Powered by Discuz! ...,NaN,4597
5,2017-01-15T16:46:54.575Z,NaN,www.99huizhou.com,99huizhou.com,http://www.99huizhou.com/thread-136351-2-1.html,找家谱 方姓的族人来帮忙-徽州姓氏-故园徽州网 - Powered by Discuz!,NaN,84,NaN,NaN,NaN,NaN,NaN,找家谱 方姓的族人来帮忙-徽州姓氏-故园徽州网 - Powered by Discuz! 故...,NaN,21776


In [40]:
df_final = df_no_dup[df_no_dup['host'] == 'cul.anhuinews.com']

In [41]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2296 entries, 5323 to 47575
Data columns (total 16 columns):
dt          2296 non-null object
from        0 non-null object
host        2296 non-null object
domain      2296 non-null object
url         2296 non-null object
title       2296 non-null object
t_tokens    0 non-null object
t_len       2296 non-null float64
summary     0 non-null object
image       0 non-null object
abstract    0 non-null object
a_tokens    0 non-null object
a_len       0 non-null object
content     2296 non-null object
c_tokens    0 non-null object
c_len       2296 non-null int64
dtypes: float64(1), int64(1), object(14)
memory usage: 304.9+ KB


In [42]:
articles = df_final['content'].str.decode('utf-8')
titles = df_final['title'].str.decode('utf-8')

In [47]:
#中安在线
def get_anhuinews_content(temp_str):
    t_split = temp_str.split(u'当前位置：')
    #t_split = temp_str.find(u'编辑信箱')
    #print type(t_split), t_split
    #print temp_str[206:]
    #print len(t_split), t_split[1]
    if len(t_split) >= 2 :
        t001 = t_split[1].split(u'标签：')
        print len(t001), t001[0]
        if len(t001) >= 1:
            return t001[0]
        else:
            t_split[1]
    else:
        return temp_str

for article in articles:
    print get_anhuinews_content(article)

for article in articles:
    print get_anhuinews_content(article)

for url in df_final['url']:
    print url

In [56]:
df_final = df_no_dup[['tstamp', 'domain', 'url', 'content', 'title', 'doc_len']]

In [57]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54952 entries, 0 to 62059
Data columns (total 6 columns):
tstamp     54952 non-null object
domain     54952 non-null object
url        54952 non-null object
content    54952 non-null object
title      54951 non-null object
doc_len    54952 non-null int64
dtypes: int64(1), object(5)
memory usage: 2.9+ MB


In [129]:
df_final.head()

,dt,from,host,domain,url,title,t_tokens,t_len,summary,image,abstract,a_tokens,a_len,content,c_tokens,c_len
5323,2017-01-15T04:56:06.958Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/,徽文化频道-中安在线,,28,,,,,,徽文化频道-中安在线 首页 安徽 政务 社会 娱乐 体育 Anhui...,,5472
5326,2017-01-15T05:02:08.625Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahmc/cai/index.shtml,徽文化频道-安徽名产-徽菜-中安在线,,48,,,,,,徽文化频道-安徽名产-徽菜-中安在线 首页 安徽 政务 社会 娱乐 体育...,,1850
5328,2017-01-15T05:03:45.11Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahmc/cha/index.shtml,徽文化频道-安徽名产-徽茶-中安在线,,48,,,,,,徽文化频道-安徽名产-徽茶-中安在线 首页 安徽 政务 社会 娱乐 体育...,,1830
5331,2017-01-15T05:24:54.761Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahmc/index.shtml,徽文化频道-安徽名产-中安在线,,41,,,,,,徽文化频道-安徽名产-中安在线 首页 安徽 政务 社会 娱乐 体育 ...,,3281
5333,2017-01-15T05:03:39.252Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahmc/jiu/index.shtml,徽文化频道-安徽名产-徽酒-中安在线,,48,,,,,,徽文化频道-安徽名产-徽酒-中安在线 首页 安徽 政务 社会 娱乐 体育...,,1980


In [107]:
articles = df_final['content'].str.decode('utf-8')
titles = df_final['title'].str.decode('utf-8')

idx = 0
for title in titles:
    if idx < 10:
        print 'title----', idx
        print title
        idx +=1

idx = 0
for article in articles:
    if idx < 10:
        print 'article----', idx
        print article
        idx +=1

In [108]:
from collections import Counter

In [109]:
stanford_root_dir = '/users/nanfanghu/desktop/nltk/stanford-segmenter-2014-08-27/'

In [110]:
segmenter = StanfordSegmenter(path_to_jar= stanford_root_dir + 'stanford-segmenter-3.4.1.jar', 
                              path_to_slf4j= stanford_root_dir + 'slf4j-api.jar',
                              path_to_sihan_corpora_dict= stanford_root_dir + 'data', 
                              path_to_model= stanford_root_dir + 'data/pku.gz', 
                              path_to_dict= stanford_root_dir + 'data/dict-chris6.ser.gz')

In [111]:
Baidu_CH_Stopwords_list = ['按, 按照, 俺, 俺们, 阿', 
                      '别, 别人, 别处, 别是, 别的, 别管, 别说, 不, 不仅 ,不但, 不光, 不单, 不只, 不外乎, 不如, 不妨, 不尽, 不尽然, 不得, 不怕, 不惟, 不成, 不拘, 不料, 不是, 不比, 不然, 不特, 不独, 不管, 不至于, 不若, 不论, 不过, 不问, 比方, 比如, 比及, 比, 本身, 本着, 本地, 本人, 本, 巴巴, 巴, 并, 并且, 非彼, 彼时, 彼此, 便于, 把, 边, 鄙人, 罢了, 被, 般的',
                     '此间,此次,此时,此外,此处,此地,此,才,才能,朝,朝着,从,从此,从而,除非,除此之外,除开,除外,除了,除,诚然,诚如,出来,出于,曾,趁着,趁,处在,乘,冲',
                     '等等,等到,等,第,当着,当然,当地,当,多,多么,多少,对,对于,对待,对方,对比,得,得了,打,打从,的,的确,的话,但,但凡,但是,大家,大,地,待,都,到,叮咚',
                     '而言,而是,而已,而外,而后,而况,而且,而,尔尔,尔后,尔,二来',
                     '非独,非特,非徒,非但,否则,反过来说,反过来,反而,反之,分别,凡是,凡',
                     '个,个别,固然,故,故此,故而,果然,果真,各,各个,各位,各种,各自,关于具体地说,归齐,归,根据,管,赶,跟,过,该,给,光是',
                     '或者,或曰,或是,或则,或,何,何以,何况,何处,何时,还要,还有,还是,还,后者,很,换言之,换句话说,好,后,和',
                     '即,即令,即使,即便,即如,即或,即若,继而,继后,继之,既然,既是,既往,既,尽管如此,尽管,尽,就要,就算,就是说,就是了,就是,就,据,据此,接着,经,经过,结果,及,及其,及至,加以,加之,例如,介于,几时,几,截至,极了,简言之,竟而,紧接着,距,较之,较,进而,鉴于,基于,具体说来,兼之,借傥然,今,叫,将',
                     '可,可以,可是,可见,开始,开外,况且,靠,看',
                     '来说,来自,来着,来,两者,临,类如,论,赖以,连,连同,离',
                     '莫若,莫如,莫不然,假使,假如,假若,某,某个,某些,某某,漫说,没奈何,每当,每,慢说,冒',
                     '哪个,哪些,哪儿,哪天,哪年,哪怕,哪样,哪边,哪里,那里,那边,那般,那样,那时,那儿,那会儿,那些,那么样,那么些,那么,那个,那,乃,乃至,乃至于,宁肯,宁愿,宁可,宁,能,能否,你,你们,您,拿,难道说,内,哪',
                     '凭借,凭,旁人,譬如, 譬喻',
                     '且,且不说,且说,其,其一,其中,其二,其他,其余,其它,其次,前后,前此,前者,起见,起,全部,全体,恰恰相反,岂但,却,去',
                     '若非,若果,若是,若夫,若,另,另一方面,另外,另悉,如若,如此,如果,如是,如同,如其,如何,如下,如上所述,如上,如,然则,然后,然而,任,任何,任凭,仍,仍旧,人家,人们,人,让',
                     '甚至于,甚至,甚而,甚或,甚么,甚且,什么,什么样,上,上下,虽说,虽然,虽则,虽,孰知,孰料,始而,所,所以,所在,所幸,所有,是,是以,是的,设使,设或,设若,谁,谁人,谁料,谁知,随着,随时,随后,随,顺着,顺,受到,使得,使,似的,尚且,庶几,庶乎,时候,省得,说来,首先',
                     '倘,倘使,倘或,倘然,倘若,同,同时,他,他人,他们们,她们,她,它们,它,替代,替,通过,腾', 
                     '这里,这边,这般,这次,这样,这时,这就是说,这儿,这会儿,这些,这么点儿,这么样,这么些,这么,这个,这一来,这,正是,正巧,正如,正值',
                     '万一,为,为了,为什么,为何,为止,为此,为着,无论,无宁,无,我们,我,往,望,惟其,唯有',
                     '下,向着,向使,向,先不先,相对而言,许多,像,小,些',
                     '一,一些,一何,一切,一则,一方面,一旦,一来,一样,一般,一转眼,,由此可见,由此,由是,由于,由,用来,因而,因着,因此,因了,因为,因,要是,要么,要不然,要不是,要不,要,与,与其,与其说,与否,与此同时,以,以上,以为,以便,以免,以及,以故,以期,以来,以至,以至于,以致,己,已,已矣,有,有些,有关,有及,有时,有的,沿,沿着,于,于是,于是乎,云云,云尔,依照,依据,依,余外,也罢,也好,也,又及,又,抑或,犹自,犹且,用,越是',
                     '只当,只怕,只是,只有,只消,只要,只限,再,再其次,再则,再有,再者,再者说,再说,自身,自打,自己,自家,自后,自各儿,自从,自个儿,自,怎样,怎奈,怎么样,怎么办,怎么,怎,至若,至今,至于,至,纵然,纵使,纵令,纵,之,之一,之所以,之类,着呢,着,眨眼,总而言之,总的说来,总的来说,总的来看,总之,在于,在下,在,诸,诸位,诸如,咱们,咱,作为,只,最,照着,照,直到,综上所述,贼死,逐步,遵照,遵循,针对,致,者,则甚,则',
                     '咳,哇,哈,哈哈,哉,哎,哎呀,哎哟,哗,哟,哦,哩,矣哉,矣乎,矣,焉,毋宁,欤,嘿嘿,嘿,嘻,嘛,嘘,嘎登,嘎,嗳,嗯,嗬,嗡嗡,嗡,喽,喔唷,喏,喂,啷当,啪达,啦,啥,啐,啊,唉,哼唷,哼,咧,咦,咚,咋,呼哧,呸,呵呵,呵,呢,呜呼,呜,呗,呕,呃,呀,吱,吧哒,吧,吗,吓,兮,儿,亦,了,乎']

In [112]:
def is_not_chinese_symbol(token):
    if token == u'\u201c' or token == u'\u201d':
        return False
    elif token == u'\uff0c' or token == u'\u3002':
        return False
    elif token == u'\u3001' or token == u'\u300b':
        return False
    elif token == u'\u300a' or token == u'\uff1a':
        return False
    elif token == u'\uff1b' or token == u'\uff09':
        return False
    elif token == u')' or token == u'(':
        return False
    elif token == u'\uff08' or token == u'\u4e00':
        return False
    elif token == u'-' or token == u':':
        return False
    elif token == u'\u548c' or  token == u'>': 
        return False
    elif token == u'\xb7' or  token == u'\u2014\u2014':         
        return False


    return True

In [113]:
combined_str = ''.decode('utf-8')
for words in Baidu_CH_Stopwords_list:
    combined_str = combined_str + words.decode('utf-8') 

print combined_str

In [114]:
Baidu_CH_Stopwords = combined_str.split(',')

print len(get_columns()), len(get_empty_columns())

In [ ]:
for i in range(0, len(df_final)):
    if(i < 10):
        print 'row ----', i
        new_df.loc[i]= get_empty_columns()
        t0 = df_final.loc[i]['title'].decode('utf-8')
        t1 = segmenter.segment(t0)
        t2 = t1.split(' ')
        tokens_no_stopwords = [token for token in t2 if token not in Baidu_CH_Stopwords and is_not_chinese_symbol(token)] 
        new_df.loc[i]['t_tokens'] = tokens_no_stopwords   

In [86]:
idx = 0
for title in df_final['title']:
    if(idx < 10):
        print 'row ----', idx
        new_df.loc[idx]['title']= get_empty_columns()
        t0 = title.decode('utf-8')
        t02 = t0.split('-')
        t03 = t02[:-3]
        #t01 = t0.replace ('- Powered by Discuz!', '')
        print t03[0]
        t1 = segmenter.segment(t03)
        t2 = t1.split(' ')
        tokens_no_stopwords = [token for token in t2 if token not in Baidu_CH_Stopwords and is_not_chinese_symbol(token)] 
        print tokens_no_stopwords[0]
        new_df.loc[idx]['t_tokens'] = tokens_no_stopwords
        idx += 1

row ---- 0
【视频】白岳笔会之座谈会
【
row ---- 1
这个怎么用？ 
？

row ---- 2
黄山日报：石潭摄影基地建设快速推进 
黄山
row ---- 3
故园徽州9月9日签到记录贴
故园
row ---- 4
找家谱 方姓的族人来帮忙
找
row ---- 5
踩点万安罗经博物馆
踩点
row ---- 6
《中国民间徽商研究小组章程》
中国
row ---- 7
清朝嘉道年间黟北卢氏茶商家书
清朝
row ---- 8
徽州婺源虹关詹氏墨商信札一通
徽州
row ---- 9
歙县旅沪同乡会会章
歙县


In [87]:
print idx

10


In [117]:
new_df = pd.DataFrame(columns = get_columns())

In [120]:
idx = 0
for item in df_final['content']:
    if(idx%1000 == 0):
        print 'row ----', idx, len(content)
        
    #print content
    new_df.loc[idx] = get_empty_columns()
    t0 = item.decode('utf-8')
    t1 = segmenter.segment(t0)
    t2 = t1.split(' ')
    tokens_no_stopwords = [token for token in t2 if token not in Baidu_CH_Stopwords and is_not_chinese_symbol(token)] 
    #print tokens_no_stopwords[0]
    new_df.loc[idx]['c_tokens'] = tokens_no_stopwords
    idx += 1

In [121]:
def map_get_tokens(s):
    t0 = s.decode('utf-8')
    t1 = segmenter.segment(t0)
    t2 = t1.split(' ')
    tokens_no_stopwords = [token for token in t2 if token not in Baidu_CH_Stopwords and is_not_chinese_symbol(token)] 
    return tokens_no_stopwords


In [130]:
df_final['c_tokens'] = df_final.content.map(lambda x: get_anhuinews_content(x) )

In [131]:
df_final.head(50)

,dt,from,host,domain,url,title,t_tokens,t_len,summary,image,abstract,a_tokens,a_len,content,c_tokens,c_len
5323,2017-01-15T04:56:06.958Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/,徽文化频道-中安在线,,28,,,,,,徽文化频道-中安在线 首页 安徽 政务 社会 娱乐 体育 Anhui...,徽文化频道-中安在线 首页 安徽 政务 社会 娱乐 体育 Anhui...,5472
5326,2017-01-15T05:02:08.625Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahmc/cai/index.shtml,徽文化频道-安徽名产-徽菜-中安在线,,48,,,,,,徽文化频道-安徽名产-徽菜-中安在线 首页 安徽 政务 社会 娱乐 体育...,徽文化频道-安徽名产-徽菜-中安在线 首页 安徽 政务 社会 娱乐 体育...,1850
5328,2017-01-15T05:03:45.11Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahmc/cha/index.shtml,徽文化频道-安徽名产-徽茶-中安在线,,48,,,,,,徽文化频道-安徽名产-徽茶-中安在线 首页 安徽 政务 社会 娱乐 体育...,徽文化频道-安徽名产-徽茶-中安在线 首页 安徽 政务 社会 娱乐 体育...,1830
5331,2017-01-15T05:24:54.761Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahmc/index.shtml,徽文化频道-安徽名产-中安在线,,41,,,,,,徽文化频道-安徽名产-中安在线 首页 安徽 政务 社会 娱乐 体育 ...,徽文化频道-安徽名产-中安在线 首页 安徽 政务 社会 娱乐 体育 ...,3281
5333,2017-01-15T05:03:39.252Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahmc/jiu/index.shtml,徽文化频道-安徽名产-徽酒-中安在线,,48,,,,,,徽文化频道-安徽名产-徽酒-中安在线 首页 安徽 政务 社会 娱乐 体育...,徽文化频道-安徽名产-徽酒-中安在线 首页 安徽 政务 社会 娱乐 体育...,1980
5337,2017-01-15T05:04:36.472Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/ahxj/index.shtml,徽文化频道-安徽戏剧-中安在线,,41,,,,,,徽文化频道-安徽戏剧-中安在线 首页 安徽 政务 社会 娱乐 体育 ...,徽文化频道-安徽戏剧-中安在线 首页 安徽 政务 社会 娱乐 体育 ...,1966
5342,2017-01-15T05:02:37.075Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/huix/index.shtml,徽文化频道-徽学-中安在线,,35,,,,,,徽文化频道-徽学-中安在线 首页 安徽 政务 社会 娱乐 体育 An...,徽文化频道-徽学-中安在线 首页 安徽 政务 社会 娱乐 体育 An...,1863
5347,2017-01-15T05:02:48.365Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/mr/index.shtml,徽文化频道-名人-中安在线,,35,,,,,,徽文化频道-名人-中安在线 首页 安徽 政务 社会 娱乐 体育 An...,徽文化频道-名人-中安在线 首页 安徽 政务 社会 娱乐 体育 An...,1669
5349,2017-01-15T05:27:51.262Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/system/2008/01/15/001...,马兰 徽文化频道 中安在线,,35,,,,,,马兰 徽文化频道 中安在线 首页 安徽 政务 社会 娱乐 体育 An...,马兰 徽文化频道 中安在线 首页 安徽 政务 社会 娱乐 体育 An...,3157
5350,2017-01-15T05:46:31.705Z,,cul.anhuinews.com,anhuinews.com,http://cul.anhuinews.com/system/2008/01/15/001...,程小君 徽文化频道 中安在线,,38,,,,,,程小君 徽文化频道 中安在线 首页 安徽 政务 社会 娱乐 体育 A...,程小君 徽文化频道 中安在线 首页 安徽 政务 社会 娱乐 体育 A...,2615


In [122]:
df_final['c_tokens'] = df_final.content.map(lambda x: map_get_tokens(x) )

KeyboardInterrupt: 

In [ ]:
df_final['t_tokens'] = df_final.title.map(lambda x: map_get_tokens(x) )

In [132]:
ii0 = 5326  #69, 219, 318, 598, 1108?, 6809, 6918

In [133]:
#t100 = df.loc[219]['content'].decode('utf-8')
#t100 = df.loc[109]['content'].decode('utf-8')  #no content
#t100 = df.loc[69]['content'].decode('utf-8')
#t100 = df.loc[318]['content'].decode('utf-8')
t100 = df.loc[ii0]['content']


In [135]:
print df.loc[ii0]['title']

In [568]:
print df.loc[ii0]['url'].decode('utf-8')


In [563]:
print t100.encode('utf-8')

In [562]:
print len(t101)

In [550]:
print get_anhuinews_content(t100)

In [520]:
#安徽文化网
def get_ahage_content(temp_str):
    t_split = temp_str.split(u'点击: 次')
    #print len(t_split), t_split[1]
    if len(t_split) >= 2 :
        t001 = t_split[1].split(u'顶一下')
        #print len(t001), t001[0]
        if len(t001) >= 1:
            return t001[0]
        else:
            return t_split[0]
    else:
        return ''

get_ahage_content(t100)

In [471]:
#故园徽州
def get_gyhz_content(temp_str):
    t_split = temp_str.split('|')
    if len(t_split) < 2:
        t001 = t_split[1].split(u'关于我们 - 官方微博 - 联系我们')
        if len(t001) >= 1:
            return t001[0]
        else:
            return t_split[0]
    else:
        for i in range(0, len(t_split)):
            print 'row ---', i 
            t001 = t_split[i]
            t002 = t001.split(u'阅读模式')
            if len(t002) >= 2:
                #print len(t002), 'pure content ---', t002[1]
                t003 = t002[1].split(u'注册会员 x')
                t004 = t003[1].split(u'本主题由')
                if len(t004) >= 2:
                    print 'got something', len(t002)
                    return t004[0]
                else:
                    return ''
    return ''

In [480]:
print get_gyhz_content(t100)

In [472]:
print ret

   黄梅戏龙凤奇缘（韩再芬主演） 本视频转自网络，只为传播黄梅文化之目的，若有侵权之处，请联系本站编辑。 


In [ ]:
dddddd


if len(t101) < 2:
        t001 = t101[0].split(u'关于我们 - 官方微博 - 联系我们')
        if len(t001) >= 1:
        print t001[0]
    else:
        print t101[0]
else:
    for i in range(0, len(t101)):
        print 'row ---', i 
        t001 = t101[i]
        t002 = t001.split(u'阅读模式')
        if len(t002) >= 2:
            #print len(t002), 'pure content ---', t002[1]
            t003 = t002[1].split(u'注册会员 x')
            t004 = t003[1].split(u'本主题由')
            print t004[0]


In [256]:
print t101[0]

阜阳颍泉区“三大平台”推进服务型党组织建设-安徽群众网-中安在线 站内搜索： 在线投稿 中央精神 


new_df['dt'] = df_final['tstamp']
new_df['from'] = df_final['from']
new_df['title'] = df_final['title']
new_df['t_len'] = df_final['t_len']
new_df['body'] = df_final['body']
new_df['b_len'] = df_final['b_len']
new_df['domain'] = df_final['domain']

    

In [3]:
i = 2000
fname = 'bosonNLP-segment-' + str(i-2000) + '-' + str(i) + '.csv' 
print fname

bosonNLP-segment-0-2000.csv
